In [9]:
import json
import csv
import random
import re

#The source CSV file
sourcedoc = "FilmClipAndQuestionsCopy.csv"
#The Film Clips file
clipdoc = "testFilmClips.json"
#The Quizzes file
quizdoc = "testQuizzes.json"
#The Quiz Questions file
quizquestiondoc = "testQuizQuestions.json"

#Input code currently disabled.
inputEnable = False
if inputEnable == True:
    print("Please include the correct file extensions in the document names.")
    print("Please enter the file name of the CSV.")
    sourcedoc=input()
    print("Please enter the file name of the filmclip output.")
    clipdoc=input()
    print("Please enter the file name of the quizzes output.")
    quizdoc=input()
    print("Please enter the file name of the quiz question output.")
    quizquestiondoc=input()

def csvToJson(sourcedoc, clipdoc, quizdoc, quizquestiondoc):
    with open (sourcedoc, "r") as f:
        reader=csv.reader(f)
        maxChoices=4 #Maximum number of answer choices per question
        count=maxChoices #Index of current answer choice - value used for determining which dictionary to add to
        correctanswer=maxChoices #Index of correct answer - value used for catching errors
        clipfile = True #If False, the current row is a piece of question info.
        data = [] #Film clips array
        data2 = [] #Quizzes array
        data3 = [] #Quiz questions array
        numQuestions = 0
        numClips = 0
        nextQIndex = 0 #Index of the current question
        rows = []
        for row in reader:
            rows.append(row)
            if row[0]=="question-text": #This signals the start of a new question.
                count=0
                data3.append({"id": str(random.randint(1000000000, 9999999999))}) #Random 10-digit question ID
                data3[numQuestions]["type"] = "MULTI-CHOICE"
                clipfile = False #The following rows are question info.
            if row[0]=="clip-title": #This signals the start of a new film clip.
                data.append({"id": str(random.randint(1000000000, 9999999999))}) #Random 10-digit film clip ID
            if row[0]=="end of quiz": #This signals the end of a film clip.
                data[numClips]["quiz"] = str(random.randint(1000000000, 9999999999)) #Random 10-digit quiz ID
                data2.append({"id": data[numClips]["quiz"]}) #Adds ID from above to the quizzes array
                data2[numClips]["questions"] = [] #Questions array
                for i in range(nextQIndex, numQuestions): #From current question index to total number of questions
                    data2[numClips]["questions"].append(data3[i]["id"]) #Questions for current film clip added to array
                    nextQIndex=nextQIndex+1
                numClips=numClips+1
            if row[0]=="question-tags": #This signals the end of a question.
                count=maxChoices #Reset
                if clipfile == False: #Current row is question info
                    data3[numQuestions]["correctAnswerIndex"] = correctanswer
                    if row[1]=="": #If tags are empty:
                        data3[numQuestions]["tags"] = [] #Empty array
                    else:
                        data3[numQuestions]["tags"] = row[1].split(";") #Tags separated by semicolons are put into an array
                    data3[numQuestions]["pointValue"] = 1
                    numQuestions=numQuestions+1
                    clipfile=True #No longer dealing with question info
                    correctanswer=maxChoices #Reset
            else:
                if row[0]!="": #Blank cells are ignored
                    if count<maxChoices:
                        """
                        The value of count is set to maxChoices at the beginning of the code and at the end of a question,
                        and it is set to 0 at the beginning of a new question. If its value is less than maxChoices,
                        the current row is a piece of question information.
                        """
                        if row[0]=="question-text":
                            data3[numQuestions]["text"] = row[1]
                            data3[numQuestions]["choices"] = [] #Array of answer choices
                        else: #Current row is an answer choice
                            count=count+1 #Index of current answer choice incremented
                            data3[numQuestions]["choices"].append(row[1]) #Answer choices added to array
                        if row[0]=="correct":
                            correctanswer=count-1 #Index of correct answer set to current index
                    else:
                        if row[0]!="end of quiz":
                            if row[0]=="clip-tags":
                                if row[1]=="": #If tags are empty:
                                    data[numClips]["tags"] = [] #Empty array
                                else:
                                    data[numClips]["tags"] = row[1].split(";") #Tags separated by semicolons are put into an array
                            else:
                                if numClips<len(data):
                                    data[numClips][row[0]] = row[1] #Add film clip information to film clip dictionary
                                else:
                                    raise ValueError("All clips must have a title.")
                                    """
                                    If a clip-title cell was not reached, a new dictionary was not appended
                                    to the film clips array, making its length too short.
                                    """

        numDesc=0 #Number of clip descriptions
        numThumb=0 #Number of clip thumbnails
        numImg=0 #Number of clip imgs
        numSrc=0 #Number of clip srcs
        numClipTags=0 #Number of clip tags
        numText=0 #Number of question texts
        allUrlsValid=True
        regex = re.compile(
            r'^(?:http|ftp)s?://' # http:// or https://
            r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
            r'localhost|' #localhost...
            r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
            r'(?::\d+)?' # optional port
            r'(?:/?|[/?]\S+)$', re.IGNORECASE)
        for i in rows:
            if i[0]=="description":
                numDesc=numDesc+1
            if i[0]=="thumbnail":
                numThumb=numThumb+1
            if i[0]=="img":
                numImg=numImg+1
            if i[0]=="src":
                numSrc=numSrc+1
            if i[0]=="clip-tags":
                numClipTags=numClipTags+1
            if i[0]=="question-text":
                numText=numText+1
            if i[0]=="thumbnail" or i[0]=="img" or i[0]=="src":
                if (re.match(regex, i[1]) is not None) == False: #entry is not a valid URL
                    allUrlsValid=False
        if not numDesc==numThumb==numImg==numSrc==numClipTags==numClips:
            raise ValueError("All film clips must have a description, thumbnail, img, src, and tags.")
            """
            If these values are not all equal, at least one clip is missing some information.
            """
        if numText>numQuestions:
            raise ValueError("All questions must have tags.")
            """
            If a question-tags cell is not reached, numQuestions is not incremented.
            This will cause numQuestions to be less than the actual number of questions,
            which should be equal to the number of question texts.
            """
        else:
            if numText<numQuestions:
                raise ValueError("All questions must have a question text.")
            """ 
            If no question is missing its tags, numQuestions will be incremented every time
            and therefore be equal to the actual number of questions. If the number of question
            texts is smaller than this, at least one question has no text.
            """
        if allUrlsValid == False:
            raise ValueError("Thumbnail, img, and src must all be URLs.")
        for i in data:
            if len(i)>8:
                raise ValueError("All questions must have a question text and no more than " + str(maxChoices) + " choices.")
                """
                If question-text is never reached, count will never be set to 0 and will therefore still be
                equal to maxChoices, so the question's choices will be added to the film clip dictionary instead.
                If a question has too many choices, count will reach the value of maxChoices before it has gotten
                through all the choices, so the remaining choices will end up in the film clip dictionary.
                These will both cause the film clip dictionary to have a length longer than the expected 8.
                """
            else:
                if len(i)<8:
                    raise ValueError("All film clips must have a description, thumbnail, img, src, and tags.")
                    """
                    If a dictionary in the film clips array is shorter than the expected 8, something must be missing.
                    """
        for i in data3:
            if i["correctAnswerIndex"] >= maxChoices: #i["correctAnswerIndex"] is equal to correctanswer.
                raise ValueError("All questions must have a correct answer.")
                """
                The value of correctanswer is initialized to maxChoices for every question and is only changed
                when a "correct" cell is reached. When changed, the value of correctanswer is always less than
                that of maxChoices. If this is not the case, then correctanswer was never changed, which means
                that a "correct" cell was never reached.
                """

    with open (clipdoc, "w", encoding="utf-8") as f:
        f.write(json.dumps(data, indent=4))
    with open (quizdoc, "w", encoding="utf-8") as f:
        f.write(json.dumps(data2, indent=4))
    with open (quizquestiondoc, "w", encoding="utf-8") as f:
        f.write(json.dumps(data3, indent=4))
    

In [10]:
csvToJson(sourcedoc, clipdoc, quizdoc, quizquestiondoc)